In [1]:
import pandas as pd
import os
import csv
import numpy as np
from matplotlib import pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
from scipy.interpolate import UnivariateSpline
from scipy.optimize import curve_fit
from scipy import interpolate, optimize

In [3]:
from Experimentos import Experimentos

[Errno 2] No such file or directory: '.\\Data\\5cc50C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\10cc60C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc60C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc59C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc63C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc62C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc62C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc24C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\10cc24C - flir.csv'


In [4]:
#experimentos sin FLIR: 11
#experimentos con FLIR: 22
for i, exp in enumerate(Experimentos):
    print(i, exp, (exp.flir is not None))

0 5cc50C.csv False
1 1cc54C.csv False
2 25cc54C.csv False
3 10cc60C.csv False
4 25cc60C.csv False
5 1cc59C.csv False
6 1cc47C.csv True
7 1cc55C.csv True
8 25cc58C.csv True
9 25cc70C.csv True
10 10cc67C.csv True
11 10cc59C.csv True
12 5cc63C.csv False
13 5cc59C.csv True
14 10cc49C.csv True
15 25cc62C.csv False
16 1cc62C.csv False
17 5cc53C.csv True
18 5a25cc56C.csv True
19 5cc57C.csv True
20 25cc50C.csv True
21 10cc55C.csv True
22 1cc24C.csv False
23 10cc57C.csv True
24 10cc24C.csv False
25 5cc55C.csv True
26 5cc58C.csv True
27 1cc48C.csv True
28 1cc51C.csv True
29 5cc48C.csv True
30 10cc50C.csv True
31 1cc54,7C.csv True
32 1cc52C.csv True


In [35]:
Experimentos[19].param.head()

,densidad manjar,modelo 3D,volumen total teorico,t amb,termocupla,caudal,angulo inclinacion,NOMBRE,Obs,Exp #,...,fecha,min,max,framei,framef,result: largo final,result: largo total,result: ancho max final,result: espesor max final,result: distancia espesor final
0,1240,NaN,0.00143,25,no,"5,00E-06",10.1,5cc57C,NaN,20,...,22/07/2016 0:00,29,49,23,1360,0.35,0.4,0.28,0.02,0.25


In [5]:
#FUNCION Q DEVUELVE RANGO [1-7] SEGUN TEMP MAXIMA DEL EXP
def rango_temperatura(temp):
    if temp < 40:
        return 1
    if temp < 45:
        return 2
    if temp < 50:
        return 3
    if temp < 55:
        return 4
    if temp < 60:
        return 5
    if temp < 65:
        return 6
    if temp < 70:
        return 7

In [6]:
#lista para armar graficos de PERFIL agrupados por caudal
#es una lista donde en cada value hay un diccionario por experimento

listaporcaudal = []

for exp in Experimentos:
    holder = {} #es un diccionario 
    holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
    holder['espesor'] = exp.perfil['perfil: espesor']#toda la columna
    holder['largo'] = exp.perfil['perfil: distancia']#toda la columna
    holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
    holder['angulo'] = exp.param['angulo inclinacion'][0]
    holder["rango temperatura"] = rango_temperatura(exp.param["t max"][0])
    holder['uso'] = exp.param['uso manjar'][0]

    listaporcaudal.append(holder)

In [15]:
#devuelve el valor que le pido recorriendo la lista creada arriba
#for i, exp in enumerate(listaporcaudal):
    #print(i,(exp['angulo']))

In [7]:
#lista para armar graficos de ANCHO VS ESPESOR agrupados por caudal
#es una lista donde en cada value hay un diccionario por experimento

listaporanchoyesp = []

for exp in Experimentos:
    holder = {} #es un diccionario 
    holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
    holder['espesor'] = exp.param['result: espesor max final']#toda la columna
    holder['ancho'] = exp.param['result: ancho max final']#toda la columna
    holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
    holder['angulo'] = exp.param['angulo inclinacion'][0]
    holder["rango temperatura"] = rango_temperatura(exp.param["t max"][0])

    listaporanchoyesp.append(holder)

In [8]:
for x in listaporcaudal:
    print(0,x['caudal'])

0 5e-06
0 1e-06
0 2.5e-05
0 1e-05
0 2.5e-05
0 1e-06
0 1e-06
0 1e-06
0 2.5e-05
0 2.5e-05
0 1e-05
0 1e-05
0 5e-06
0 5e-06
0 1e-05
0 2.5e-05
0 1e-06
0 5e-06
0 1.7e-05
0 5e-06
0 2.5e-05
0 1e-05
0 1e-06
0 1e-05
0 1e-05
0 5e-06
0 5e-06
0 1e-06
0 1e-06
0 5e-06
0 1e-05
0 1e-06
0 1e-06


In [9]:
#se arman grupos segun el valor del caudal
from itertools import groupby
count = 0

#key es la lista donde estan los diccionarios de exp y DF
#group es que se agrupa segun el valor de 'caudal' en este caso

for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    count += 1
    #for exp in group:
        #muestra el nombre del grupo y los elementos que tiene
        #print(exp['nombre']) 

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [10]:
count

5

In [11]:
#EJES GRAFICOS PERFILES SEGUN CAUDAL
maximoejelargo = max([exp['largo'].max() for exp in listaporcaudal])
minimoejelargo = min([exp['largo'].min() for exp in listaporcaudal])
maximoejeespesor = max([exp['espesor'].max() for exp in listaporcaudal])
minimoejeespesor = min([exp['espesor'].min() for exp in listaporcaudal])

In [12]:
#EJES GRAFICOS ANCHOS MAXIMOS SEGUN CAUDAL
maximoejeancho = max([exp['ancho'].max() for exp in listaporanchoyesp])
minimoejeancho = min([exp['ancho'].min() for exp in listaporanchoyesp])
maximoejeespesor = max([exp['espesor'].max() for exp in listaporanchoyesp])
minimoejeespesor = min([exp['espesor'].min() for exp in listaporanchoyesp])

In [13]:
#GENERA PATCHES DE COLORES SEGUN TEMPERATURA
import matplotlib.patches as mpatches

pat1 = mpatches.Patch(color='C1', label='temp < 40')
pat2 = mpatches.Patch(color='C2', label='temp < 45')
pat3 = mpatches.Patch(color='C3', label='temp < 50')
pat4 = mpatches.Patch(color='C4', label='temp < 55')
pat5 = mpatches.Patch(color='C5', label='temp < 60')
pat6 = mpatches.Patch(color='C6', label='temp < 65')
pat7 = mpatches.Patch(color='C7', label='temp < 70')

numerocolores = 7
colors = ['pat{}'.format(i+1) for i in range(numerocolores)]

print(colors)

['pat1', 'pat2', 'pat3', 'pat4', 'pat5', 'pat6', 'pat7']


In [14]:
#CASI LISTO!!! FALTA LEYENDA LINEA PUNTEADA Y CONTINUA
#5 GRAFICOS DE PERFIL AGRUPADOS POR CAUDAL Y COLOREADOS SEGUN TEMEPRATURA
fig = plt.figure()
axes = [fig.add_subplot(3,2,i+1) for i in range(count)]
counter = 0

for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    ax = axes[counter]
    ax.set_xlim(minimoejelargo, maximoejelargo)
    ax.set_ylim(0, maximoejeespesor)
    ax.set_title('{}cc'.format(key*1000000))
    counter += 1
    
    for exp in group:
        #print("C{}".format(exp["rango temperatura"]))
        if exp['angulo'] == 15:
            ax.plot(exp['largo'], exp['espesor'], color='C{}'.format(exp['rango temperatura']), 
                    linestyle= '--')
        else:
            ax.plot(exp['largo'], exp['espesor'], color='C{}'.format(exp['rango temperatura']))
            
linea1= ax.plot([],[], label='15°', linestyle='--', color='black')
linea2= ax.plot([],[], label='10°', color='black')      
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
plt.gca().add_artist(legend1)
fig.show()

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [15]:
#LISTOOOO
#5 GRAFICOS DE PERFIL AGRUPADOS POR CAUDAL Y COLOREADOS SEGUN TEMEPRATURA 
#SOLO PRIMER USO DEL MANJAR
fig = plt.figure()
axes = [fig.add_subplot(3,2,i+1) for i in range(count)]
counter = 0



for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    ax = axes[counter]
    ax.set_xlim(minimoejelargo, maximoejelargo)
    ax.set_ylim(0, maximoejeespesor)
    ax.set_title('{}cc'.format(key*1000000))
    counter += 1
    linea1= ax.plot([],[], label='15°', linestyle='--', color='black')
    linea2= ax.plot([],[], label='10°', color='black')
    
    for exp in group:
        #print("C{}".format(exp["rango temperatura"]))
        if exp['uso'] == 1: 
            if exp['angulo'] == 15:
                ax.plot(exp['largo'], exp['espesor'], color='C{}'.format(exp['rango temperatura']), 
                        linestyle= '--')
            else:
                ax.plot(exp['largo'], exp['espesor'], color='C{}'.format(exp['rango temperatura']))
        
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
plt.gca().add_artist(legend1)
fig.show()

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [20]:
#LISTO!!!
#1 GRAFICO DE ESPESOR VS ANCHO AGRUPADOS POR CAUDAL Y COLOREADOS X TEMPERATURA
fig, ax = plt.subplots()
counter = 0
marqer = ['o', 'v', '^', '<', '>']
plotmarqer = []

from matplotlib import markers

contadorkey = 0

for key, group in groupby(sorted(listaporanchoyesp, key=lambda x: x['caudal']), 
                          key=lambda x: x['caudal']):
    print(key)
    ax.set_xlim(minimoejeancho-0.02, maximoejeancho+0.02)
    ax.set_ylim(maximoejeespesor-0.02, maximoejeespesor+0.001)
    markadorkey = marqer[contadorkey]
    contadorkey += 1
    
    for i, exp in enumerate(group):
        if exp['angulo'] == 15:
            l = ax.plot(exp['ancho'], exp['espesor'], color='C{}'.format(exp['rango temperatura']), 
                           marker=markadorkey)
            #l.set_markeredgecolor('Black')
        else: 
            l = ax.plot(exp['ancho'], exp['espesor'], color='C{}'.format(exp['rango temperatura']), 
                           marker=markadorkey)
        if i == 0:
            plotmarqer.append(ax.scatter([], [], color='black', marker=markadorkey, label=exp['caudal']))

legendcolors2 = [pat1, pat2, pat3, pat4, pat5, pat6, pat7]

legend1 = plt.legend(handles=plotmarqer, loc= 3)
plt.legend(handles=legendcolors2, loc=2)
plt.gca().add_artist(legend1)
plt.show()

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [16]:
#lista para armar figures de velocidad segun Q
listadinamica = []
#es una lista donde en cada value hay un diccionario por experimento
#ANOTAR EN UNA HOJA COMO SE VE Y USA ESTO PLS
for exp in Experimentos:
    holder = {} #es un diccionario 
    holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
    holder['tiempo'] = exp.dinamica['avance: tiempo']#toda la columna
    holder['distancia'] = exp.dinamica['avance: distancia desde punto eyeccion']#toda la columna
    holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
    holder['angulo'] = exp.param['angulo inclinacion'][0]
    holder["rango temperatura"] = rango_temperatura(exp.param["t max"][0])
        
    listadinamica.append(holder)

In [27]:
#len(listadinamica)
#    if exp.param['t max'][0] < 40:
#        holder['rango temperatura'] = 7
#        continue
#    if exp.param['t max'][0] < 45:
#        holder['rango temperatura'] = 1
#        continue
#    if exp.param['t max'][0] < 50:
#        holder['rango temperatura'] = 2
#        continue
#    if exp.param['t max'][0] < 55:
#        holder['rango temperatura'] = 3
#        continue
#    if exp.param['t max'][0] < 60:
#        holder['rango temperatura'] = 4
#        continue
#    if exp.param['t max'][0] < 65:
#        holder['rango temperatura'] = 5
#    if exp.param['t max'][0] < 70:
#        holder['rango temperatura'] = 6  


In [26]:
#FALTA: LEYENDA 25CC DENTRO DEL GRAFICO Y ARREGLAR LEYENDA TIPO LINEA Y ANGULO
#empezamo el ploteo de todas las ##dinamicas agrupados segun caudal
fig = plt.figure()
axes = []

#este plot hace cuantos subplot necesite segun un contador que da el numero de
#grupos generados en el groupby
for x in range(count):
    axes.append(fig.add_subplot(2, 3, x + 1))

#UNA COLUMNA DE GRAFICOS CON VARIAS FILAS SEGUN GRUPO
#se generan los ejes de cada plot
#i es el indice de lista y luego cada value en lista (???)
for i, (key, group) in enumerate(groupby(sorted(listadinamica, key=lambda x: x['caudal']), key=lambda x: x['caudal'])):
    print(i, key)

    for exp in group:
        axes[i].legend([key])
        if exp['angulo'] == 15:
            axes[i].loglog(exp['tiempo'], exp['distancia'], color='C{}'.format(exp['rango temperatura']),
                           linestyle='--')
        else:
            axes[i].loglog(exp['tiempo'], exp['distancia'], color='C{}'.format(exp['rango temperatura']))
        axes[i].legend([key])
            
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.gca().add_artist(legend1)

linea1= ax.plot([],[], label='15°', linestyle='--', color='black')
linea2= ax.plot([],[], label='10°', color='black')       
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
fig.show()

0 1e-06
1 5e-06
2 1e-05
3 1.7e-05
4 2.5e-05


In [15]:
fig= plt.plot(Experimentos[6].dinamica['avance: tiempo'], Experimentos[6].dinamica['avance: largo total flujo'], marker = 'x')

In [47]:
Experimentos[30].dinamica.tail()

,Unnamed: 0,avance: tiempo,avance: distancia desde punto eyeccion,avance: distancia hacia atrás,avance: largo total flujo,avance: ancho maximo
44,44,5441.333333,0.360,0.074,0.434,0.305
45,45,5967.333333,0.362,0.074,0.436,0.305
46,46,9159.333333,0.368,0.074,0.442,0.305
47,47,9986.333333,0.368,0.074,0.442,0.305
48,48,76919.333333,0.368,0.074,0.442,0.305


In [27]:
exp = Experimentos[30]
exp

10cc50C.csv

In [28]:
exp.dinamica.tail()

,Unnamed: 0,avance: tiempo,avance: distancia desde punto eyeccion,avance: distancia hacia atrás,avance: largo total flujo,avance: ancho maximo,volumen_emitido,anchoporlargo,area_avance,espesor
44,44,5441.333333,0.360,0.074,0.434,0.305,0.001493,0.033092,0.103910,0.021546
45,45,5967.333333,0.362,0.074,0.436,0.305,0.001493,0.033245,0.104389,0.021447
46,46,9159.333333,0.368,0.074,0.442,0.305,0.001493,0.033702,0.105826,0.021156
47,47,9986.333333,0.368,0.074,0.442,0.305,0.001493,0.033702,0.105826,0.021156
48,48,76919.333333,0.368,0.074,0.442,0.305,0.001493,0.033702,0.105826,0.021156


In [29]:
(exp.dinamica['avance: ancho maximo']/2)*(exp.dinamica['avance: largo total flujo']/2)

0     0.000000
1     0.000475
2     0.000750
3     0.001085
4     0.001575
5     0.002175
6     0.002650
7     0.003190
8     0.003875
9     0.004521
10    0.005250
11    0.005957
12    0.007000
13    0.008075
14    0.009000
15    0.010335
16    0.011531
17    0.012916
18    0.014318
19    0.015660
20    0.017000
21    0.018395
22    0.019845
23    0.021128
24    0.022294
25    0.023004
26    0.024215
27    0.025628
28    0.026550
29    0.027300
30    0.028143
31    0.028595
32    0.028896
33    0.029747
34    0.030125
35    0.030424
36    0.030805
37    0.031415
38    0.031949
39    0.032330
40    0.032559
41    0.032940
42    0.033092
43    0.033092
44    0.033092
45    0.033245
46    0.033702
47    0.033702
48    0.033702
dtype: float64

In [30]:
rho_manjar = 1340
kilos_manjar = 2
mitad = 2
pi = 3.14

for exp in Experimentos:
    exp.dinamica['volumen_emitido'] = exp.param['caudal'][0]*exp.dinamica['avance: tiempo']
    exp.dinamica['anchoporlargo'] = (exp.dinamica['avance: ancho maximo']/mitad)*(exp.dinamica['avance: largo total flujo']/mitad)
    exp.dinamica['area_avance'] = exp.dinamica['anchoporlargo']*pi
    volumen_total = kilos_manjar/rho_manjar
    exp.dinamica['volumen_emitido'][exp.dinamica['volumen_emitido'] > volumen_total] = volumen_total
    exp.dinamica['espesor'] = 3*exp.dinamica['volumen_emitido']/(2*exp.dinamica['area_avance'])

C:\Users\CCVV\Anaconda3b\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [31]:
exp.dinamica.head()

,Unnamed: 0,avance: tiempo,avance: distancia desde punto eyeccion,avance: distancia hacia atrás,avance: largo total flujo,avance: ancho maximo,volumen_emitido,anchoporlargo,area_avance,espesor
0,0,0.000000,0.00,0.000,0.000,0.000,0.000000,0.000000,0.000000,NaN
1,1,5.500000,0.01,0.010,0.020,0.035,0.000005,0.000175,0.000550,0.015014
2,2,32.833333,0.02,0.020,0.040,0.050,0.000033,0.000500,0.001570,0.031369
3,3,69.500000,0.03,0.032,0.062,0.070,0.000069,0.001085,0.003407,0.030600
4,4,113.166667,0.04,0.037,0.077,0.085,0.000113,0.001636,0.005138,0.033039


In [21]:
#lista para armar figures de velocidad segun Q
listaespesor = []
#es una lista donde en cada value hay un diccionario por experimento
#ANOTAR EN UNA HOJA COMO SE VE Y USA ESTO PLS
for exp in Experimentos:
    holder = {} #es un diccionario 
    holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
    holder['tiempo'] = exp.dinamica['avance: tiempo']#toda la columna
    holder['espesor'] = exp.dinamica['espesor']#toda la columna
    holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
    holder['angulo'] = exp.param['angulo inclinacion'][0]
    holder["rango temperatura"] = rango_temperatura(exp.param["t max"][0])
        
    listaespesor.append(holder)

In [25]:
fig= plt.plot(Experimentos[8].dinamica['avance: tiempo'], Experimentos[8].dinamica['espesor'], marker = 'x')

In [32]:
#Graficos: ESPESOR vs Tiempo

fig = plt.figure()
axes = []

for x in range(count):
    axes.append(fig.add_subplot(2, 3, x + 1))

for i, (key, group) in enumerate(groupby(sorted(listaespesor, key=lambda x: x['caudal']), key=lambda x: x['caudal'])):
    print(i, key)

    for exp in group:
        axes[i].legend([key])
        if exp['angulo'] == 15:
            axes[i].loglog(exp['tiempo'], exp['espesor'], color='C{}'.format(exp['rango temperatura']),
                           linestyle='--')
        else:
            axes[i].loglog(exp['tiempo'], exp['espesor'], color='C{}'.format(exp['rango temperatura']))
        axes[i].legend([key])
            
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.gca().add_artist(legend1)

linea1= ax.plot([],[], label='15°', linestyle='--', color='black')
linea2= ax.plot([],[], label='10°', color='black')       
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
fig.show()

0 1e-06
1 5e-06
2 1e-05
3 1.7e-05
4 2.5e-05


In [34]:
#lista para armar figures de velocidad segun Q
listaancho = []
#es una lista donde en cada value hay un diccionario por experimento
#ANOTAR EN UNA HOJA COMO SE VE Y USA ESTO PLS
for exp in Experimentos:
    holder = {} #es un diccionario 
    holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
    holder['tiempo'] = exp.dinamica['avance: tiempo']#toda la columna
    holder['ancho'] = exp.dinamica['avance: ancho maximo']#toda la columna
    holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
    holder['angulo'] = exp.param['angulo inclinacion'][0]
    holder["rango temperatura"] = rango_temperatura(exp.param["t max"][0])
        
    listaancho.append(holder)

In [35]:
#Graficos: ANCHO VS Tiempo
fig = plt.figure()
axes = []

for x in range(count):
    axes.append(fig.add_subplot(2, 3, x + 1))

for i, (key, group) in enumerate(groupby(sorted(listaancho, key=lambda x: x['caudal']), key=lambda x: x['caudal'])):
    print(i, key)

    for exp in group:
        axes[i].legend([key])
        if exp['angulo'] == 15:
            axes[i].loglog(exp['tiempo'], exp['ancho'], color='C{}'.format(exp['rango temperatura']),
                           linestyle='--')
        else:
            axes[i].loglog(exp['tiempo'], exp['ancho'], color='C{}'.format(exp['rango temperatura']))
        axes[i].legend([key])
            
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
plt.gca().add_artist(legend1)

linea1= ax.plot([],[], label='15°', linestyle='--', color='black')
linea2= ax.plot([],[], label='10°', color='black')       
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
fig.show()

0 1e-06
1 5e-06
2 1e-05
3 1.7e-05
4 2.5e-05
